# import the necessary package

In [1]:
import pandas as pd
import pickle
import random
pd.set_option('display.max_columns',100)
%matplotlib inline


# load the dataset

**News Articles ** 

This is the "raw" newsdeed data that we want to try to match with projects. If you run into a protocol error try the '_py27.pkl' version 

In [2]:
newsData = pd.read_pickle('../Data/Feedly_Processed_DF_cleaned.pkl')

Note that the third item's `title` field looks to have a lot of content - there is likely cleaning needed on some of these fields. 

In [4]:
newsData.head(3)

article_id                                              title  \
10900   eebb9702  India, World Bank sign financing agreement for...   
4268    6832ce57  Rs 40000-crore development projects in limbo i...   
1663    30f8f65e  https://www.the-american-interest.com/2018/01/...   

                                                     url  \
10900  http://www.abplive.in/business/india-world-ban...   
4268   http://www.moneycontrol.com/news/business/econ...   
1663   https://www.the-american-interest.com/2018/01/...   

                    feed_label content           published  \
10900     NEWS WB- All Streams     NaN 2017-12-21 09:22:12   
4268   NEWS AIIB - All Streams     NaN 2017-12-10 09:40:00   
1663    NEWS AFDB- All Streams     NaN 2018-01-03 12:21:54   

                                                 summary  \
10900  <table border="0" cellspacing="3" cellpadding=...   
4268   <table border="0" cellspacing="3" cellpadding=...   
1663   <table border="0" cellspacing="3" cellpadding=...   

                                            article_text  \
10900  New Delhi [India], Dec 20 (ANI): A financing a...   
4268   Development projects worth more than Rs 40,000...   
1663   Ten Lessons\n\nDevelopment with Chinese Charac...   

                                        article_keywords  article_text_len  \
10900  [institutes, india, skill, financing, training...              1031   
4268   [development, crore, andhra, eaps, state, proj...              4390   
1663   [transitions, university, chinese, united, dev...               575   

      top_lang  
10900       en  
4268        en  
1663        en

** Projcets Data ** 

This is the "raw" unmatched project data 

In [6]:
projectInfo = pd.read_csv('../Data/EWS_Published Project_Listing_DD.csv',encoding='ISO-8859-1')

In [7]:
projectInfo.head(3)

EWS ID      ProjectNumber  Published Bank Risk Rating Project Status  \
0   29164  AFDB-P-TN-BB0-007  Published                U       Proposed   
1   29166  AFDB-P-SZ-HAA-001  Published                U       Approved   
2   29931      IADB-UR-T1100    Pending                C       Approved   

                                             EWS URL  Detailed Analysis URL  \
0  https://ews.rightsindevelopment.org/projects/p...                    NaN   
1  https://ews.rightsindevelopment.org/projects/p...                    NaN   
2  https://ews.rightsindevelopment.org/projects/u...                    NaN   

                                        Project Name City  Country Count  \
0                         TUNISIA FERTILIZER PROJECT  NaN              1   
1  LINE OF CREDIT TO SWAZILAND DEVELOPMENT FINANC...  NaN              1   
2  Supporting INEFOP in Improving Labor Training ...  NaN              1   

   Country 1 Country 2 Country 3 Country 4 Country 5 Country 6 Country 7  \
0    Tunisia       NaN       NaN       NaN       NaN       NaN       NaN   
1  Swaziland       NaN       NaN       NaN       NaN       NaN       NaN   
2    Uruguay       NaN       NaN       NaN       NaN       NaN       NaN   

  Country 8 Country 9 Country 10 Country 11 Country 12 Borrower or Client  \
0       NaN       NaN        NaN        NaN        NaN                NaN   
1       NaN       NaN        NaN        NaN        NaN                NaN   
2       NaN       NaN        NaN        NaN        NaN                NaN   

   Private Actor Count Private Actor 1 Private Actor 2 Private Actor 3  \
0                  0.0             NaN             NaN             NaN   
1                  0.0             NaN             NaN             NaN   
2                  0.0             NaN             NaN             NaN   

  Private Actor 4 Private Actor 5 Private Actor 6 Private Actor 7  \
0             NaN             NaN             NaN             NaN   
1             NaN             NaN             NaN             NaN   
2             NaN             NaN             NaN             NaN   

  Private Actor 8  Private Actor 9  Private Actor 10  Private Actor 11  \
0             NaN              NaN               NaN               NaN   
1             NaN              NaN               NaN               NaN   
2             NaN              NaN               NaN               NaN   

   Private Actor 12  Private Actor 13  Private Actor 14  Private Actor 15  \
0               NaN               NaN               NaN               NaN   
1               NaN               NaN               NaN               NaN   
2               NaN               NaN               NaN               NaN   

   Bank Count                                  Bank 1 Bank 2 Bank 3 Bank 4  \
0           1         African Development Bank (AFDB)    NaN    NaN    NaN   
1           1         African Development Bank (AFDB)    NaN    NaN    NaN   
2           1  Inter-American Development Bank (IADB)    NaN    NaN    NaN   

   Bank 5  Sector Count                  Sector 1            Sector 2  \
0     NaN             1  Agriculture and Forestry                 NaN   
1     NaN             2                   Finance  Industry and Trade   
2     NaN             0                       NaN                 NaN   

  Sector 3 Sector 4 Sector 5 Sector 6 Sector 7 Last Edited Date Scraped  \
0      NaN      NaN      NaN      NaN      NaN      9/4/17      8/15/17   
1      NaN      NaN      NaN      NaN      NaN      9/4/17      8/15/17   
2      NaN      NaN      NaN      NaN      NaN         NaN      10/3/17   

  Date Disclosed Board Date  \
0       12/13/01   12/13/01   
1       12/13/01    5/12/17   
2       12/31/99    7/16/13   

                                          Source URL  Project Cost  \
0  http://www.afdb.org/en/projects-and-operations...           NaN   
1  http://www.afdb.org/en/projects-and-operations...          4.76   
2  http://www.iadb.org/en/projects/project-d

**Labeled Projcet Data ** 

There isn't much of this data - so it is wise to treat this as a test/validation set for any algorithm you develop. Perhaps you can use a small sample of this data to build intuition for designing your algorithm. 

In [8]:
projectsLabeled = pd.read_csv('../Data/Labeled_Data/projects.csv',encoding='ISO-8859-1')

In [10]:
projectsLabeled.head(3)

article_id published                                              title  \
0    10f9ed2   1/11/18  ADB Provides Support for Three Infrastructure ...   
1   c0eece9b   5/13/18  ADB Helps Inaugurate New Power Distribution Ne...   
2   d1d79dd8   2/20/18  ADB Provides $360 Million for Rolling Stock to...   

                                                 url              feed_label  \
0  http://moderndiplomacy.eu/2018/01/11/adb-provi...  NEWS ADB - All Streams   
1  http://feedproxy.google.com/~r/adb_news/~3/2My...  NEWS ADB - All Streams   
2  http://feedproxy.google.com/~r/adb_news/~3/v9s...  NEWS ADB - All Streams   

                                 ProjectNumber  \
0  ADB-41123-015, ADB-48158-001, ADB-41435-053   
1                                ADB-47282-001   
2                                ADB-50312-003   

                                    EWS Project Name  \
0  Road Network Improvement Project (formerly Sec...   
1  Energy Supply Improvement Investment Program (...   
2  Railway Rolling Stock Operations Improvement P...   

                                       EWS hyperlink  Matched  
0  https://ewsdata.rightsindevelopment.org/projec...        1  
1  https://ewsdata.rightsindevelopment.org/projec...        1  
2  https://ewsdata.rightsindevelopment.org/projec...        1

# Possible Places to Start 

1. Extract features from the News Articles and the raw Projects data (bank, keywords, country etc). From news article with extract the banks from feed_label. feed_label is from news article. 

2. Maybe start with title and banks - see if you are getting any matches that make sense.

# When you want to Test your algorithm 

* The projects in the labeled data should also present in the projects data. 
* WATCH OUT : Some articles are linked to multiple projects so you need to decide on how to deal with before you just join the files
* ALSO : The labeled data inlcudes articles that don't match projects (Articles that should be classified as having NO Match) - you can see this if the "Matched" field = 0. 

In [17]:
projectsLabeled[['article_id','ProjectNumber']].head(20)

article_id                                ProjectNumber
0     10f9ed2  ADB-41123-015, ADB-48158-001, ADB-41435-053
1    c0eece9b                                ADB-47282-001
2    d1d79dd8                                ADB-50312-003
3     f0d65e5                                ADB-50410-001
4    4a557358                                ADB-50410-001
5    817a9fee                                ADB-50410-001
6    1d18e02b                                ADB-50410-001
7    5ae8d202                                ADB-50410-001
8    e9482ff2                                ADB-50410-001
9    c2a956dd                                          NaN
10   9fd5c398                                          NaN
11   b2d42591                                          NaN
12   6c25cda0                                  AIIB-000011
13     f08d78                                  AIIB-000015
14   21bcdf5b           AIIB-000015, AIIB-0003, AIIB-00057
15    cb7b1ba           AIIB-000015, AIIB-0003, AIIB-00057
16   d71af9e4           AIIB-000015, AIIB-0003, AIIB-00057
17   ed93f7da           AIIB-000015, AIIB-0003, AIIB-00057
18   def9e94e                                  AIIB-000019
19   b37f386d                                  AIIB-000020